In [85]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from stop_word_list import stop_words_lst
import time
import re
from tqdm import tqdm_notebook

In [2]:
from addr_dict import state_dict, substate_dict

# brand dictionary module

##### 본 모듈은 점포 이름이 ~점 으로 끝나는 data를 가지고 
##### 공백을 기준으로 나눠진 제일 앞글자를 list로 만들어 
##### db에 있는 data로 brand list를 만드는 모듈입니다.

--------------------------------------------------------------------

 ~점에서 앞글자는 지역 명칭을 의미하며 
 맨 앞글자는 브랜드라고 가정합니다.


### db, table 및 dictionary만들 column name 설정

In [4]:
table_name='eums.MEUMS_COLL_CARD_MER'

In [80]:
col_name='CARD_USE_HIS'

### ~점으로 끝나는 것중 지역 이름이 아닌것 

In [83]:
remove_end_list=['백화점','전문점','가맹점','대리점','면세점','수리점','안경점','서점','양복점','양품점','단란주점','한복점','반점','노래주점','양장점','주점','매점','편의점','정육점']

### 본 모듈을 이용해 출력된 list 중 실제 brand가 아닌 이름들. 
- brand list 의 질을 위해  ban_list의 update 필요

In [81]:
ban_list=['24시','25시','21세기','1박2일','OK','SK','BF','cafe','AK','가가','강남','강릉','',' '
          ,'cafe.','CC','JJ','coffee','구미','국제','JW','PC','YK','강화','경남','고향',
          '카페','피자','네이쳐','분식의','쥬스','KCP결제','에스','커피','불닭발','바른','한국','조선',
          '쉐프의','백년','통큰','버터','생고기','장보는','오렌지','해피','러브','아이스','마이','메가']

### table data count

In [5]:
date_n_df=pd.read_sql('select count(1) from {}'.format(table_name),input_db)

In [6]:
data_n=date_n_df['count(1)'].values[0]

In [8]:
def delete_stopwords(s, stop_words_lst):
    """Clean strings by removing unwanted stopwords.
    
    Parametersb
    ----------
    s : string. string to delete stopwords

    stop_words_lst : list. list of stopwords 


    Example
    -------

    stop_words_lst = ["(주)", "(유)", "(사)", "(의)", "(재)",
                    "주식회사", "유한회사", "사단법인", "의료법인", "재단법인",
                    "(사단)", "(유한)", "(의료)", "(법인)", "(재단)",
                    "(주식회사)", "(유한회사)", "(사단법인)", "(의료법인)", "(재단법인)", 
                    "[아이행복]", "온누리상품권", "무이자"]

    Returns
    -------
    string
    """

    for sw in stop_words_lst:
        s = s.replace(sw, ' ')

    return s

### data 정제 및 brand 추출

In [35]:
case=[]
case_id_dict={}

for idx in tqdm_notebook(range(0,(int)(data_n/10000)+1)): 

    df=pd.read_sql('select * from {} limit {},10000'.format(table_name,idx*10000),input_db)
    
    # data 정제 
    # data 중 ~점으로 끝나고, 띄어쓰기가 되어있는 것
    # remove_end_list 로 끝나는 것 제거
    
    df=df[~df[col_name].str.contains(r'(중개)$')]    
    df=df[df[col_name].str.contains(r'[점]$')]
    string= string="\\"+"\\b$|\\".join(n for n in remove_end_list) + "\\b"
    df=df[~df[col_name].str.contains(r'{}'.format(string))]
    df[col_name]=df[col_name].str.replace(r'\(','')
    df[col_name]=df[col_name].str.replace(r'\)','')
    df[col_name]=df[col_name].apply(delete_stopwords,stop_words_lst = stop_words_lst)
    
    df[col_name+'_len']=df[col_name].str.split()
    df[col_name+'_len']=df[col_name+'_len'].apply(lambda x :len(x))
    df=df[df[col_name+'_len']>1]
    
    # 이미 case에 들어가 있는 brand이름이 string에 있다면 제거
    
    for k in case:
        temp_df=df[df[col_name].str.contains(r"^({})".format(k))]
        add=[]
        if k in case_id_dict:
            add=case_id_dict[k]
        
        temp=list(temp_df['ID'])
        for kd in temp:
            add.append(kd)
        case_id_dict[k]=add
        
        df=df[~df[col_name].str.contains(r'^({})'.format(k))]
    
    

    # 맨 첫번째 단어를 가져와 case에 추가. 
    
    while 0<len(df):
        
        spt=df.iloc[0][col_name].split(' ')
        target=spt[0]
        
        #ban_list에 속한다면 제외
        # 한글, 알파벳, 숫자 이외의 단어 포함 시 제외
        
        if len(target)==1 or target in ban_list or re.findall('[^가-힣ㄱ-ㅎA-Za-z0-9]',target):
            pass
        else:
            case.append(target)
            temp_df=df[df[col_name].str.contains(r"^({})".format(target))]
            add=[]
            for i in range(0,len(temp_df)):
                add.append(temp_df.iloc[i]['ID'])

            case_id_dict[target]=add

            df=df[~df[col_name].str.contains(r'^({})'.format(target))]

        df=df.iloc[1:]
            
    print(len(case))
    print('%d'%(idx*10000))


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


processing time : 1.39
184
0


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


processing time : 1.71
336
10000
processing time : 2.10
480
20000
processing time : 2.56
611
30000
processing time : 2.88
735
40000
processing time : 2.78
828
50000
processing time : 3.25
941
60000
processing time : 3.36
1033
70000
processing time : 3.63
1135
80000
processing time : 3.81
1223
90000
processing time : 3.93
1295
100000
processing time : 4.22
1374
110000
processing time : 4.47
1453
120000
processing time : 4.72
1542
130000
processing time : 4.83
1620
140000
processing time : 5.09
1711
150000
processing time : 5.20
1785
160000
processing time : 5.30
1857
170000
processing time : 5.47
1927
180000
processing time : 6.04
2008
190000
processing time : 5.75
2069
200000
processing time : 6.34
2143
210000
processing time : 6.35
2218
220000
processing time : 6.56
2287
230000
processing time : 6.66
2351
240000
processing time : 7.09
2420
250000
processing time : 7.26
2490
260000
processing time : 7.27
2549
270000
processing time : 7.37
2613
280000
processing time : 7.74
2694
290000


In [36]:
len(case)

3791

#### '주'를 포함하는 case이외의 case에서 '주'를 제외한 단어를 가지고 있는 data가 있다면 제외

#### ex) 주신세계백화점 | 신세계백화점 => 주신세계백화점 제외

In [37]:
filter_list_ju=[]
for idx in case_id_dict:
    if len(case_id_dict[idx])>10:
        if re.findall('(주)',idx):
            filter_list_ju.append(idx)

In [38]:
filter_test=[]
for idx in case_id_dict:
    if len(case_id_dict[idx])>10:
        if re.findall('(^주)',idx):
            for kda in filter_list_ju:
                if re.findall('({})'.format(idx),'{}'.format(kda)):
                    filter_test.append(kda)

In [39]:
filter_not_filt=[idx for idx in filter_list_ju if idx in filter_test]

In [40]:
for idx in filter_not_filt:
    del case_id_dict[idx]

#### 위 filter에서 특정 빈도 이상 나타난 case만 brand_list에 넣음

In [60]:
brand_list=[]

In [61]:
for idx in case_id_dict:
    if len(case_id_dict[idx])>10:
        brand_list.append(idx)

In [62]:
brand_list=list(set(brand_list))

In [63]:
len(brand_list)

356

In [64]:
for idx in brand_list:
    print(idx)

부산아지매국밥
토마토도시락
가마로강정
세븐일레븐
피엠푸드
이차돌
크린토피아
미니스톱
렌즈미
VIC마켓
피자스쿨
셀렉토커피
월드크리닝
커피니
블럭제빵소
동대문
홍익돈까스
네이처컬렉션
하모니마트
쥬씨
김가네
담소사골순대
BKR버거킹
아이센스
또봉이
새마을식당
씨스페이스
롭스
돌배기집
불닭발동대문엽기떡볶이
더페이스샵
호치킨
아딸
쥬시
뚜레쥬르
원할머니보쌈
롯데쇼핑
명랑시대쌀핫도그
제일제면소
꼬마대통령
동경야시장
명랑쌀핫도그
리얼야구존
더벤티
미스터힐링
하남돼지집
애니카랜드
파리바케트
오마뎅
또아식빵
한국미니스톱
큰맘
아비꼬
엉터리생고기
신의주
파리바게뜨
비지에프리테일
365플러스
싸움의고수
지지고
써브웨이
킹콩부대찌개
메가
현대옥
더샘
총각네
아이스크림
롯데슈퍼
스피드메이트
아오리의
메가마트
코리아세븐
모모스테이크
꼬지사께
GS수퍼
한국피자헛
포차어게인
알파
이디야
롯데쇼핑주
스트라이크
계절밥상
지코바
유니클로
카페베네
뉴욕버거
이바돔감자탕
이화수전통육개장
명인만두
자연별곡
롯데리아
이철헤어커커
봉구비어
컴포즈커피
맛닭꼬
미소야
홍루이젠
BHC
한국맥도날드
씨제이
청담이상
폴바셋
베트남
한국유통
에프알엘코리아
네이처리퍼블릭
로그인
탐앤탐스
커피빈코리아
하프타임
청년떡볶이
로이드
투다리
샤브향
GS25
요거프레소
텀브커피
비비큐
홈마트
아마스빈
모노치즈
죽이야기
아웃백
바푸리
대백마트
한스델리
달콤커피
엔젤리너스
커피나무
경성주막
굽네치킨
올리브영
교촌치킨
곱창고
리챠드프로헤어
비알코리아주
더캐롯
씨유
인디오븐
이태리부대찌개
서가앤쿡
컴포즈
던킨도너츠
커피베이
예스
홍대개미
가장맛있는족발
홍익궁중전통육개장
채선당
디초콜릿커피
바른치킨
이마트
떼루와
자연드림
파파존스
치킨마루
못된고양이
투썸
더착한커피
돈돈정
피자샵
마노핀
리안헤어
홈플러스
콩심
엔제리너스
백채
마리웨일
신세계
베스트올
빚은
MARKET999
놀부보쌈
펀비어킹
포메인
봉구스밥버거
드림디포
육쌈냉면
빽다방
불닭발땡초동대문엽기떡볶이
남다른감자탕
토리헤어
헬로방방
애슐리
파스쿠찌
피자알볼로
홍콩반점
노브랜드

## brand list 적용 예시

In [68]:
filter_df=pd.read_sql('select * from {} limit {},100000'.format(table_name,0),input_db)

In [72]:
res_filtered=pd.DataFrame()
for idx in tqdm_notebook(brand_list):
    add=filter_df[filter_df[col_name].str.contains('{}'.format(idx))]
    add['brand']=idx
    res_filtered=pd.concat([res_filtered,add])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [77]:
pd.set_option('display.max_rows', 5000)

In [78]:
res_filtered[[col_name,'brand']]

,CARD_USE_HIS,brand
902,부산아지매국밥차병원사거리점,부산아지매국밥
65247,부산아지매국밥 BC카드점,부산아지매국밥
75743,부산아지매국밥,부산아지매국밥
81545,부산아지매국밥상암dmc점,부산아지매국밥
88232,부산아지매국밥상암dmc,부산아지매국밥
12697,토마토도시락용인명지대점,토마토도시락
18724,토마토도시락대구,토마토도시락
24753,토마토도시락이매점,토마토도시락
28597,토마토도시락경희대점,토마토도시락
42438,토마토도시락대구계명대점,토마토도시락
